In [19]:
!pip install pydantic[email]
!pip install fastapi

# Validação básica com Pydantic

Testando validações simples no Pydantic, teste com dados considerados bons e ruins, vendo na prática como funciona a validação

In [20]:
from enum import auto, IntFlag
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    SecretStr,
    ValidationError,
)

# define roles de usuários como flags
class Role(IntFlag):
    Author = auto()
    Editor = auto()
    Developer = auto()
    Admin = Author | Editor | Developer # O admin tem os privilégios de Author/Editor/Developer

# definição da classe User, com campos que serão validados automaticamente. Cada campo possui uma descrição documentada e pode possuir também exemplos de como é esperado aquele campo (ex: email e password)
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["example@arjancodes.com"],
        description="The email address of the user",
        frozen=True, # Impede que seja mudado depois da validação
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(default=None, description="The role of the user")

# tenta criar um User (imprime possíveis erros)
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data) # "cria o user" e valida todos os campos
        print(user) # mostra valores validados
    except ValidationError as e:
        print("User is invalid")
        for error in e.errors():
            print(error)

# teste com dados bons e ruins, para demonstrar a validação com o pydantic
def main() -> None:
    good_data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }
    bad_data = {"email": "<bad data>", "password": "<bad data>"}

    validate(good_data) # imprime Users sem erros
    validate(bad_data) # lista os erros de validação


if __name__ == "__main__":
    main()

name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '<bad data>', 'password': '<bad data>'}, 'url': 'https://errors.pydantic.dev/2.11/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '<bad data>', 'ctx': {'reason': 'An email address must have an @-sign.'}}


# Utilização de Regex para fazer validação de dados com o Pydantic

Aplicação de validações com regex no Pydantic, testando cenários bons e ruins para nomes e senhas fortes, somado a aplicação de hash SHA-256 para proteger as credenciais

In [21]:
import enum
import hashlib
import re
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$") # uso do regex para garantir uma senha forte: minimo de 8 caracteres, 1 letra maiuscula, 1 minuscula, 1 digito
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$") # uso do regex para nome conter apenas letras e com pelo menos 2 caracteres


class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

# modelo com validações customizadas (fiels_validator e model_validator)
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(
        default=None, description="The role of the user", examples=[1, 2, 4, 8]
    )

    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role: # converte para o enum Role correspondente
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v: # valida a presença de campos obrigatórios
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold(): # garante que a senha não contenha o nome do usuário
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]): # verifica se a senha é forte
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest() # armazena o hash em vez da senha em formato de texto
        return v


def validate(data: dict[str, Any]) -> None: # função de validação utilizando o modelo aprimorado
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)


def main() -> None: # teste de múltiplos cenários para demostrar os validadores funcionando
    test_data = dict(
        good_data={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )

    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()


if __name__ == "__main__":
    main()

good_data
name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid:
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

bad_data
User is invalid:
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

bad_name
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://

# Serialização de dados com o Pydantic

Customizando a serialização no Pydantic, teste de diferentes dumps JSON e exclusões de campos e implementando validações pós-modelo

In [22]:
import enum
import hashlib
import re
from typing import Any, Self
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
)

VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")


class Role(enum.IntFlag):
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

# BaseModel com validações pré e pós, além de serializers customizados
class User(BaseModel):
    name: str = Field(examples=["Example"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user", exclude=True # não inclui senha na exportação do modelo
    )
    role: Role = Field(
        description="The role of the user",
        examples=[1, 2, 4, 8],
        default=0,
        validate_default=True, # valor default também é validado
    )

    @field_validator("name")
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role: # reusa o validados anterior
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user_pre(cls, v: dict[str, Any]) -> dict[str, Any]: # as validações antereiores garantem requisitos básicos
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest() # a senha é convertida para hash antes de contruir o modelo
        return v

    @model_validator(mode="after") # demonstração de uso da validação pós-modelo
    def validate_user_post(self, v: Any) -> Self:
        if self.role == Role.Admin and self.name != "Arjan":
            raise ValueError("Only Arjan can be an admin")
        return self

    @field_serializer("role", when_used="json")
    @classmethod
    def serialize_role(cls, v) -> str: # serializa o role como nome em JSON
        return v.name

    @model_serializer(mode="wrap", when_used="json")
    def serialize_user(self, serializer, info) -> dict[str, Any]: # controla quais campos exportar no JSON
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)


def main() -> None:
    data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
        "role": "Admin",
    }
    user = User.model_validate(data)
    if user: # demonstra diversas formas de exportar o modelo
        print(
            "The serializer that returns a dict:",
            user.model_dump(),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a JSON string:",
            user.model_dump(mode="json"),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a json string, excluding the role:",
            user.model_dump(exclude=["role"], mode="json"),
            sep="\n",
            end="\n\n",
        )
        print("The serializer that encodes all values to a dict:", dict(user), sep="\n")


if __name__ == "__main__":
    main()

The serializer that returns a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'role': <Role.Admin: 4>}

The serializer that returns a JSON string:
{'name': 'Arjan', 'role': 'Admin'}

The serializer that returns a json string, excluding the role:
{'name': 'Arjan', 'email': 'example@arjancodes.com'}

The serializer that encodes all values to a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>}


# Pydantic e FastAPI

Criando API com FastAPI e Pydantic, expondo endpoints para criar, listar e buscar usuários, validando payloads e executando testes automatizados do fluxo

In [23]:
from datetime import datetime
from typing import Optional
from uuid import uuid4

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

app = FastAPI()

class User(BaseModel):
    model_config = {
        "extra": "forbid",
    }
    __users__ = [] # lista para armazenar na memória os usuários
    name: str = Field(..., description="Name of the user")
    email: EmailStr = Field(..., description="Email address of the user")
    friends: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of friends"
    )
    blocked: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of blocked users"
    )
    signup_ts: Optional[datetime] = Field(
        default_factory=datetime.now, description="Signup timestamp", kw_only=True
    )
    id: UUID4 = Field(
        default_factory=uuid4, description="Unique identifier", kw_only=True
    )

    @field_serializer("id", when_used="json")
    def serialize_id(self, id: UUID4) -> str:
        return str(id) # conversão para json


@app.get("/users", response_model=list[User])
async def get_users() -> list[User]:
    return list(User.__users__) # retorna todos os usuários cadastradsos


@app.post("/users", response_model=User)
async def create_user(user: User):
    User.__users__.append(user) # validação automática (pydantic e fastapi)
    return user


@app.get("/users/{user_id}", response_model=User)
async def get_user(user_id: UUID4) -> User | JSONResponse: # busca pelo usuário (utilizando o ID), se o usuário não for encontrado é retornado o erro 404
    try:
        return next((user for user in User.__users__ if user.id == user_id))
    except StopIteration:
        return JSONResponse(status_code=404, content={"message": "User not found"})

# teste automatizado usando TestClient do fastapi
def main() -> None:
    with TestClient(app) as client: # criação de 5 usuários e verificação do status e conteudo
        for i in range(5):
            response = client.post(
                "/users",
                json={"name": f"User {i}", "email": f"example{i}@arjancodes.com"},
            )
            assert response.status_code == 200
            assert response.json()["name"] == f"User {i}", (
                "The name of the user should be User {i}"
            )
            assert response.json()["id"], "The user should have an id"

            user = User.model_validate(response.json())
            assert str(user.id) == response.json()["id"], "The id should be the same"
            assert user.signup_ts, "The signup timestamp should be set"
            assert user.friends == [], "The friends list should be empty"
            assert user.blocked == [], "The blocked list should be empty"

        response = client.get("/users") # verifica listagem de usuários
        assert response.status_code == 200, "Response code should be 200"
        assert len(response.json()) == 5, "There should be 5 users"

        # testa criação e recuperação de usuário
        response = client.post(
            "/users", json={"name": "User 5", "email": "example5@arjancodes.com"}
        )
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "The name of the user should be User 5"
        )
        assert response.json()["id"], "The user should have an id"

        user = User.model_validate(response.json())
        assert str(user.id) == response.json()["id"], "The id should be the same"
        assert user.signup_ts, "The signup timestamp should be set"
        assert user.friends == [], "The friends list should be empty"
        assert user.blocked == [], "The blocked list should be empty"

        response = client.get(f"/users/{response.json()['id']}")
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "This should be the newly created user"
        )

        response = client.get(f"/users/{uuid4()}") # teste do endpoint (método GET) com um ID inválido
        assert response.status_code == 404
        assert response.json()["message"] == "User not found", (
            "We technically should not find this user"
        )

        response = client.post("/users", json={"name": "User 6", "email": "wrong"}) # testa a validação de email ao criar usuário com tipo de email inválido
        assert response.status_code == 422, "The email address is should be invalid"


if __name__ == "__main__":
    main()

In [24]:
from datetime import datetime
from typing import Optional
from typing import List
from uuid import uuid4

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

app = FastAPI()

#criando o objeto usuário com as propriedades necessárias
class User(BaseModel):
    model_config = {
        "extra": "forbid",
    }
    __users__ = [] # lista para armazenar na memória os usuários
    name: str = Field(..., description="Name of the user")
    email: EmailStr = Field(..., description="Email address of the user")
    friends: List[UUID4] = Field(
        default_factory=list, max_items=500, description="List of friends"
    )
    blocked: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of blocked users"
    )
    signup_ts: Optional[datetime] = Field(
        default_factory=datetime.now, description="Signup timestamp", kw_only=True
    )
    id: UUID4 = Field(
        default_factory=uuid4, description="Unique identifier", kw_only=True
    )

    @field_serializer("id", when_used="json") # conversão para json
    def serialize_id(self, id: UUID4) -> str:
        return str(id)


@app.get("/users", response_model=list[User]) # retorna todos os usuários cadastradsos
async def get_users() -> list[User]:
    print("get_users")
    return list(User.__users__)


@app.post("/users", response_model=User) # validação automática (pydantic e fastapi)
async def create_user(user: User):
    print(user)
    User.__users__.append(user)
    return user


@app.get("/users/{user_id}", response_model=User) # busca pelo usuário (utilizando o ID), se o usuário não for encontrado é retornado o erro 404
async def get_user(user_id: UUID4) -> User | JSONResponse:
    try:
        print(user_id)
        return next((user for user in User.__users__ if user.id == user_id))
    except StopIteration:
        return JSONResponse(status_code=404, content={"message": "User not found"})

# teste automatizado usando TestClient do fastapi junto das validações do pydantic
def main() -> None:
    with TestClient(app) as client: # criação de 5 usuários e verificação do status e conteudo
        for i in range(5):
            response = client.post(
                "/users",
                json={"name": f"User {i}", "email": f"example{i}@arjancodes.com"}, # requisição do tipo post passando usuário para a api
            )

            assert response.status_code == 200
            assert response.json()["name"] == f"User {i}", (
                "The name of the user should be User {i}"
            )
            assert response.json()["id"], "The user should have an id"

            user = User.model_validate(response.json())
            assert str(user.id) == response.json()["id"], "The id should be the same"
            assert user.signup_ts, "The signup timestamp should be set"
            assert user.friends == [], "The friends list should be empty"
            assert user.blocked == [], "The blocked list should be empty"

        response = client.get("/users") # verifica listagem de usuários
        assert response.status_code == 200, "Response code should be 200"
        assert len(response.json()) == 5, "There should be 5 users"

        # testa criação e recuperação de usuário
        response = client.post(
            "/users", json={"name": "User 5", "email": "example5@arjancodes.com"}
        )
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "The name of the user should be User 5"
        )
        assert response.json()["id"], "The user should have an id"

        user = User.model_validate(response.json())
        assert str(user.id) == response.json()["id"], "The id should be the same"
        assert user.signup_ts, "The signup timestamp should be set"
        assert user.friends == [], "The friends list should be empty"
        assert user.blocked == [], "The blocked list should be empty"

        response = client.get(f"/users/{response.json()['id']}")
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "This should be the newly created user"
        )

        response = client.get(f"/users/{uuid4()}") # teste do endpoint (método get) com um ID inválido
        assert response.status_code == 404
        assert response.json()["message"] == "User not found", (
            "We technically should not find this user"
        )

        response = client.post("/users", json={"name": "User 6", "email": "wrong"}) # testa a validação de email ao criar usuário com tipo de email inválido
        assert response.status_code == 422, "The email address is should be invalid"


if __name__ == "__main__":
    main()

name='User 0' email='example0@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 6, 9, 2, 20, 53, 871059) id=UUID('ec3ae781-02fa-4d5d-b3f8-b8ad269b83d4')
name='User 1' email='example1@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 6, 9, 2, 20, 53, 874973) id=UUID('8f949dd8-8f02-49f1-887d-68dc2cfca20b')
name='User 2' email='example2@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 6, 9, 2, 20, 53, 877469) id=UUID('e11cbe1e-d888-4d72-8fb8-027727fe9f62')
name='User 3' email='example3@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 6, 9, 2, 20, 53, 879868) id=UUID('7cb92766-7fc4-4b3c-ab56-b2f5ec7faf50')
name='User 4' email='example4@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 6, 9, 2, 20, 53, 882279) id=UUID('4ebf378d-ca0b-4f0c-9758-c3e71af357bb')
get_users
name='User 5' email='example5@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 6, 9, 2, 20, 